# C-Town year simulation using weeks

This notebook can be used to run a C-Town simulation for 9 months in a "week-by-week" (weeks of 10-days) basis
The notebook reads from .csv files to define the tank level initial conditions for the week and the demand patterns for that week. 
The tank initial levels are stored in "tank_initial_conditions" and the demand points are defined in "starting_demand_points" and "three_years_demands_ctown"

In [1]:
# load module
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import wntr
import wntr_utils
from sklearn.preprocessing import MinMaxScaler
from random import shuffle
import csv
import random

In [2]:
week_index = 1
total_demands = pd.read_csv('three_year_demands_ctown.csv', index_col=0)
demand_starting_points = pd.read_csv('starting_demand_points.csv', index_col=0)
initial_tank_levels = pd.read_csv('tank_initial_conditions.csv', index_col=0)
network_delays = pd.read_csv('network_links_delay.csv', index_col=0)
network_losses = pd.read_csv('network_loss_small.csv', index_col=0)

In [3]:
# load EPANET model
inp_file = 'ctown_map.inp'
wn = wntr.network.WaterNetworkModel(inp_file)

C:\Users\Azulm\Anaconda3\lib\site-packages\wntr\epanet\io.py:2085: UserWarning: Not all curves were used in "ctown_map.inp"; added with type None, units conversion left to user
  warnings.warn('Not all curves were used in "{}"; added with type None, units conversion left to user'.format(self.wn.name))


In [4]:
week_index = 1-1
week_start = demand_starting_points.iloc[week_index][0]

In [10]:
week_demands = total_demands.loc[week_start:week_start+239:]

In [7]:
# Replace the demand patterns with the random patterns for this week
names, patterns =[],[]
for name, pat in wn.patterns():  
    pat.multipliers = week_demands[name].values.tolist()

In [8]:
wn.options.time.duration=len(week_demands.values)

In [11]:
initial_tank_levels.iloc[week_index]

T1    5.622006
T2    1.373781
T3    2.109871
T4    3.680605
T5    1.466813
T6    1.915251
T7    4.015033
Name: 0, dtype: float64

In [12]:
# Replace the initial tank levels with the values stored in the .csv file
for i in range(1,8):    
    wn.get_node('T' + str(i)).init_level = float(initial_tank_levels.iloc[week_index]['T'+ str(i)])

In [12]:
# Simulate hydraulics
sim = wntr.sim.WNTRSimulator(wn,mode='PDD')

In [13]:
# this would run the simulation. 
# Mohammad, using this notebook as base you could change the notebook to run all the 38 weeks. Notice that the variable week_index 
# is the one controlling which week is being simulated
results_new = sim.run_sim()

NameError: name 'sim' is not defined

In [15]:
pressure_results = results_new.node['pressure']
pressure_results.to_csv('test_ctown_week_1_wntr.csv')

In [28]:
network_delays.iloc[0]['r1']

21.208797452631494

In [27]:
network_losses.iloc[0]['r1']

0.2448607788241941

In [31]:
def get_node_list_by_type(list, type):
    result = []
    for node in list:
        if wn.get_node(node).node_type == type:
            result.append(str(node))
    return result

In [33]:
def get_link_list_by_type(list, type):
    result = []
    for link in list:
        if wn.get_link(link).link_type == type:
            result.append(str(link))
    return result

In [46]:
node_list      = list(wn.node_name_list)
link_list = list(wn.link_name_list)
tank_list      = get_node_list_by_type(node_list, 'Tank')
pump_list      = get_link_list_by_type(link_list, 'Pump')
valve_list     = get_link_list_by_type(link_list, 'Valve')

print(tank_list)
print(pump_list)
print(valve_list)

['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7']
['PU1', 'PU2', 'PU3', 'PU4', 'PU5', 'PU6', 'PU7', 'PU8', 'PU9', 'PU10', 'PU11']
['v1', 'V45', 'V47', 'V2']


In [47]:
column_list = tank_list.copy()
column_list.extend(pump_list)
column_list.extend(valve_list)
print(column_list)

['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'PU1', 'PU2', 'PU3', 'PU4', 'PU5', 'PU6', 'PU7', 'PU8', 'PU9', 'PU10', 'PU11', 'v1', 'V45', 'V47', 'V2']


In [65]:
wn.get_link('V2').status

<LinkStatus.Closed: 0>

In [67]:
values = []
for tank in tank_list:
    values.append(wn.get_node(tank).level)

for pump in pump_list:
    if type(wn.get_link(pump).status) is int:
        values.append(wn.get_link(pump).status)
    else:
        values.append(wn.get_link(pump).status.value)

for valve in valve_list:
    if type(wn.get_link(valve).status) is int:
        values.append(wn.get_link(valve).status)
    else:
        values.append(wn.get_link(valve).status.value)                
print(values)        
print(len(values))
print(len(column_list))

[5.62200632906611, 1.3737812430682368, 2.1098712996780336, 3.680605078293752, 1.4668132696702116, 1.9152510119241555, 4.015032667850306, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0]
22
22


In [72]:
last_values = pd.DataFrame(data = values)
last_values = last_values.T
last_values.columns = column_list
print(last_values)
last_values.to_csv('last_values_test.csv')

         T1        T2        T3        T4        T5        T6        T7  PU1  \
0  5.622006  1.373781  2.109871  3.680605  1.466813  1.915251  4.015033  0.0   

   PU2  PU3  ...  PU6  PU7  PU8  PU9  PU10  PU11   v1  V45  V47   V2  
0  1.0  0.0  ...  0.0  0.0  0.0  0.0   0.0   0.0  2.0  2.0  2.0  0.0  

[1 rows x 22 columns]


In [83]:
loaded_values = pd.read_csv('last_values_test.csv')

for tank in tank_list:
    wn.get_node(tank).init_level = float(loaded_values.iloc[0][tank])

for pump in pump_list:
        wn.get_link(pump).status = float(loaded_values.iloc[0][pump])

for valve in valve_list:
        wn.get_link(valve).status = float(loaded_values.iloc[0][valve])        